In [1]:
import numpy as np
import torch
import plotly.graph_objects as go
from binary_btsp_network_torch import BinaryBTSPNetworkTorch  # 你定义的类
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

net = BinaryBTSPNetworkTorch(
    n_input=20000,
    n_memory=30000,
    fw=0.6,   # 连接密度
    fq=0.0025,  # 学习翻转概率
    fp=0.005,  # 输入模式稀疏度
    threshold=10
)

# 生成训练模式
patterns = net.make_sparse_patterns(n_patterns=200)

# 训练前记忆
memory_before = net.forward(patterns[1])
print("训练前 CA1 激活向量（部分）:", memory_before[:50].cpu().numpy())  # 打印前50个元素
print("训练前 CA1 激活神经元数量:", memory_before.sum().item())

# 执行一次 BTSP 更新（训练）
net.btsp_update(patterns[1])

# 训练后记忆
memory_after = net.forward(patterns[1])
print("训练后 CA1 激活向量（部分）:", memory_after[:50].cpu().numpy())
print("训练后 CA1 激活神经元数量:", memory_after.sum().item())

训练前 CA1 激活向量（部分）: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
训练前 CA1 激活神经元数量: 30000
训练后 CA1 激活向量（部分）: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
训练后 CA1 激活神经元数量: 30000


In [2]:
import matplotlib.pyplot as plt

# ------------------------------
# 计算每个 CA1 神经元的输入加和
# ------------------------------
# 训练前
input_pattern = patterns[1]
ca1_input_before = (net.weights.float() * input_pattern.float()).sum(dim=1).cpu().numpy()

# 训练后
net.btsp_update(input_pattern)  # 如果你还没更新
ca1_input_after = (net.weights.float() * input_pattern.float()).sum(dim=1).cpu().numpy()

# ------------------------------
# 绘制直方图
# ------------------------------
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.hist(ca1_input_before, bins=50, color='skyblue', edgecolor='black')
plt.title("训练前 CA1 接收连接强度分布")
plt.xlabel("输入加和")
plt.ylabel("频率")

plt.subplot(1,2,2)
plt.hist(ca1_input_after, bins=50, color='salmon', edgecolor='black')
plt.title("训练后 CA1 接收连接强度分布")
plt.xlabel("输入加和")
plt.ylabel("频率")

plt.tight_layout()
plt.show()


AttributeError: 'BinaryBTSPNetworkTorch' object has no attribute 'weights'